In [6]:
from pymilvus import connections, utility, MilvusException, FieldSchema, CollectionSchema, DataType, Collection, loading_progress
connections.connect(host="localhost", port="19530", db_name='default')

In [17]:
fields = [
    FieldSchema(name="Data_Id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="Dataset_File", dtype=DataType.VARCHAR,max_length=255),
    FieldSchema(name="Collected", dtype=DataType.VARCHAR,max_length=100),
    FieldSchema(name="Variable", dtype=DataType.VARCHAR,max_length=100),
    FieldSchema(name="Label", dtype=DataType.VARCHAR,max_length=255),
    FieldSchema(name="Description", dtype=DataType.FLOAT_VECTOR, dim=1024),
]

collection_schema = CollectionSchema(fields, description="Example_collection")

doc_data = Collection("Milvus_Test_2", collection_schema)


In [18]:
#Index on vector
def Create_Index(collection_Name: str, vector_name: str, index_par: dict):
    try:
        collection = Collection(collection_Name)
        collection.create_index(field_name = vector_name, index_params = index_par)
    except Exception as e:
        print(e)
    else:
        print("Index created")

index_params = {
    "index_type" : "IVF_FLAT",
    "metric_type" : "L2",
    "params" : {"nlist":128}  ,
}

Create_Index("Milvus_Test_2", "Description", index_params) 


Index created


In [19]:
#insert function 
#NOT TEST YET
def Insert_vector(id_list: list, data_file_list: list, collected_list: list, variable_list: list, label_list: list, embeddings_Description, Collection):
    try:
        collection = Collection
 
        for id_item, data_file_item, collected_item, variable_item, label_item, embeddings_Description in zip(id_list, data_file_list, collected_list, variable_list, label_list, embeddings_Description):
            entities = [[id_item], [data_file_item], [collected_item], [variable_item], [label_item], [embeddings_Description]]
            collection.insert(entities)
            print(f"编号{id_item}插入完毕")
        collection.flush()
 
    except Exception as e:
        print(e)

In [30]:
def Insert_vector2(id_list: list, data_file_list: list, collected_list: list, variable_list: list, label_list: list, embeddings_Description, Collection):
    try:
        collection = Collection
 
        for id in id_list:
            index = id - 1
            entities = [[id_list[index]], [data_file_list[index]], [collected_list[index]], [variable_list[index]], [label_list[index]], [embeddings_Description[index]]]
            collection.insert(entities)
            print(f"编号{id}插入完毕")
    except Exception as e:
        print(e)

In [ ]:
import time
import pandas as pd
import json

from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection
from pymilvus.model.hybrid import BGEM3EmbeddingFunction

bge_m3_ef = BGEM3EmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    device='cpu', # Specify the device to use, e.g., 'cpu' or 'cuda:0'
    use_fp16=False # Specify whether to use fp16. Set to `False` if `device` is `cpu`.
)

Collection_Name = 'Milvus_Test_2'
collection = Collection(Collection_Name)

file_path = 'data_1.csv'
df = pd.read_csv(file_path)
print(df.columns)

id_list = df.index.tolist()
id_plus_one = [x + 1 for x in id_list]
dataset_file_data = df["Dataset File"].tolist()
collected_data = df.Collected.tolist()
variable_data = df.Variable.tolist()
label_data = df.Label.tolist()

df['Description'] = df['Description'].fillna('no description')
description_data = df['Description'].tolist()


print(f"list最后一个元素是{id_plus_one[-1]}")
#embedding
Description_embedding = bge_m3_ef.encode_documents(description_data)["dense"]

print(type(Description_embedding[0][0]))
# import ipdb;ipdb.set_trace()

# d_embed = Description_embedding.tolist()
start_time = time.time()
try:
    # insert_data = [id_plus_one, dataset_file_data, collected_data, variable_data, label_data, Description_embedding]
    Insert_vector2(id_plus_one, dataset_file_data, collected_data, variable_data, label_data, Description_embedding, Collection = collection)
    # mr = collection.insert(insert_data)
except Exception as e:
    print(f"插入数据时发生错误：{e}")
end_time = time.time()
print(f"插入数据用时为: {end_time - start_time}秒, 平均一条的插入时间为: {(end_time - start_time)/len(id_list)}")
print(f"总共有: {len(id_list)}条数据")

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

Index(['Dataset File', 'Collected', 'Variable', 'Label', 'Description'], dtype='object')
list最后一个元素是7166


Inference Embeddings:   1%|█▋                                                                                                                                                          | 5/448 [00:06<09:36,  1.30s/it]

In [29]:
#search
try:
    res = collection.query(
        expr = " Data_Id == 1",
        output_fields=["Collected"]
        # collection_name="Milvus_Test_2",
        # filter="1 < Data_Id < 5",
        # output_fields=["Collected"]
    )
    
    print(res)
except Exception as e:
    print(f"插入数据时发生错误：{e}")    

[{'Collected': 'Baseline', 'Data_Id': 1}]
